# Clearing address from dataframe origin - Tratando base de dados dos endereços do dataframe original. 

In [ ]:
!pip install geopandas  

In [4]:
!pip install shapely

In [5]:
import matplotlib 
import pandas as pd
import geopandas as gpd

In [6]:
from shapely.geometry import Point, Polygon  

In [7]:
from geopandas import geoseries 

- csv file generated from building.ipynb notebook. -- Arquivo csv gerado a partir do notebook building.ipynb. 

In [ ]:
ppi_address_ajust = '/content/address_ppi.csv'
ppi_address_ajust = pd.read_csv(ppi_address_ajust, encoding='ISO-8859-1')
ppi_address_ajust.head()

In [ ]:
ppi_address_ajust.drop('Unnamed: 0', axis = 1)

In [ ]:
ppi_address_ajust.dtypes

In [ ]:
ppi_address_ajust.values

In [12]:
ppi_address_ajust['ppi_ajust_cols'] = ppi_address_ajust['Empreendimento'] + '/' + ppi_address_ajust['UF']

In [ ]:
ppi_address_ajust

In [14]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

- 'brazil_states' 

# Tools Geocode, for point location 
 - return GeometryCollection empty

In [ ]:
# gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'Brazil')

In [ ]:
# ppi_address_ajust['Geom'] = gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'brazil_states')['geometry']

In [ ]:
# ppi_address_ajust

In [ ]:
# ppi_address_ajust.loc[ppi_address_ajust['Geom'] == 'GEOMETRYCOLLECTION EMPTY', 'Geom'].head()

In [ ]:
# ppi_address_ajust.drop_duplicates() # if have, < inplace = True > 

# Geocoders implementation for locations acess - implementação de acesso das localizações usando o Geocoders 
- return None

In [15]:
import geopy.geocoders

In [16]:
from geopy.geocoders import Nominatim

In [17]:
geopy.geocoders.options.default_user_agent = 'my_app/1'

In [18]:
geopy.geocoders.options.default_timeout = 7

In [19]:
geolocator = Nominatim()

In [20]:
print(geolocator.headers)

{'User-Agent': 'my_app/1'}


In [21]:
print(geolocator.timeout)

7


- Test aplication 

In [22]:
# locator = Nominatim(user_agent='myGeocoder')
location = geolocator.geocode('Aeroporto de Salvador/ BA')

In [ ]:
print(location)

In [24]:
print(location.point)

12 54m 15.1844s S, 38 20m 27.9298s W


In [25]:
location.point

Point(-12.904217899999999, -38.341091608139536, 0.0)

In [26]:
from geopy.extra.rate_limiter import RateLimiter

- Search all address from dataframe

In [27]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [28]:
ppi_address_ajust['ppi_geocode'] = ppi_address_ajust['Empreendimento'].apply(geocode)

In [ ]:
ppi_address_ajust

- plot points of address 

In [30]:
ppi_address_ajust['point'] = ppi_address_ajust['ppi_geocode'].apply(lambda loc: tuple(loc.point) if loc else None)

- ajust None's

In [31]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

In [ ]:
ppi_address_ajust

In [ ]:
ppi_address_ajust.isna()

- New position from point values not null. 

In [ ]:
ppi_address_ajust.sort_values(by='point', ignore_index=True)

In [38]:
ppi_address_ajust.index

RangeIndex(start=0, stop=191, step=1)

# Trait DF 

In [58]:
ppi_point = ppi_address_ajust

In [ ]:
ppi_point

In [ ]:
ppi_point.sort_values(by='point', ignore_index=True)

In [61]:
ppi_point = ppi_point.drop(['Unnamed: 0'], axis=1)

In [62]:
ppi_point_true = ppi_point.iloc[:30, :3]

In [ ]:
ppi_point_true

In [66]:
ppi_point_false = ppi_point.iloc[30:191, :3]

In [ ]:
ppi_point_false

In [71]:
ppi_point_false.reset_index()

,index,Empreendimento,UF,ppi_ajust_cols
0,30,STM 04  Terminal de Combustíveis em Santarém/ PA,PA,STM 04  Terminal de Combustíveis em Santarém/...
1,31,STM 05  Terminal de Combustíveis em Santarém/ PA,PA,STM 05  Terminal de Combustíveis em Santarém/...
2,32,TECON - Terminal de Contêineres de Salvador/ BA,BA,TECON - Terminal de Contêineres de Salvador/ B...
3,33,FOSPAR S.A. - Terminal de Fertilizantes no Por...,PR,FOSPAR S.A. - Terminal de Fertilizantes no Por...
4,34,PAR-01 - Terminal Portuário para Movimentação ...,PR,PAR-01 - Terminal Portuário para Movimentação ...
5,35,PAR-12 - Terminal Portuário para Movimentação ...,PR,PAR-12 - Terminal Portuário para Movimentação ...
6,36,IQI-18 - Terminal Portuário para Movimentação ...,MA,IQI-18 - Terminal Portuário para Movimentação ...
7,37,MCP-01 - Terminal Portuário para Movimentação ...,AP,MCP-01 - Terminal Portuário para Movimentação ...
8,38,TEQUIMAR - Terminal Químico de Aratu S.A. - Po...,MA,TEQUIMAR - Terminal Químico de Aratu S.A. - Po...
9,39,XXXIX - Terminal Portuário de Caramuru - Porto...,SP,XXXIX - Terminal Portuário de Caramuru - Porto...


In [74]:
location_00 = geolocator.geocode('BR-116 - Presidente Dutra	')

In [75]:
print(location_00)

Rodovia Presidente Dutra, Várzea do Palácio, Guarulhos, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 07034-900, Brasil


In [43]:
# ppi_point = ppi_address_ajust.drop(['Unnamed: 0',	'Empreendimento',	'UF',	'ppi_ajust_cols'], axis=1)

# To remember codes: 

In [ ]:
#show
# ppi_address_ajust.index.values
# ppi_address_ajust.Geom.values
# type(ppi_address_ajust.Geom.values)

# rename 
# ppi_address_ajust.rename(columns= {'Unnamed: 0': 'Rows_place'}, inplace=True )